In [1]:
%config Completer.use_jedi=False

In [2]:
import json
from copy import deepcopy

In [3]:
with open('A.json', 'r') as f:
    A = json.load(f)
with open('E.json', 'r') as f:
    E = json.load(f)
with open('I.json', 'r') as f:
    I = json.load(f)
with open('O.json', 'r') as f:
    O = json.load(f)

In [4]:
def get_vals_at_idx(json_obj, idx):
    tracks = json_obj[0]['tracks']
    vals = {}
    for elem in tracks:
        name = elem['name']
        value = elem['values'][idx]
        vals[name] = value
    return vals

In [5]:
def get_init_vals(json_obj):
    return get_vals_at_idx(json_obj, 0)

In [6]:
all_sounds = [A, E, I, O]
sound_names = ['A', 'E', 'I', 'O']

In [7]:
def areDictsEqual(dict1, dict2, eps=1e-6):
    equal = True
    for key1, val1 in dict1.items():
        val2 = 0 if not key1 in dict2 else dict2[key1]
        if not (val1 - val2 < eps):
            print(key1, val1, val2)
            equal = False
    return equal

In [8]:
for idx in range(len(all_sounds)-1):
    are_equal = areDictsEqual( get_init_vals(all_sounds[idx]), get_init_vals(all_sounds[idx+1]) )
    print(f'{sound_names[idx]} and {sound_names[idx+1]} are equal: {are_equal}')

A and E are equal: True
E and I are equal: True
LowerTeeth_M.position 0.029289955273270607 0
UpperTeeth_M.position 0.04549021273851395 0
I and O are equal: False


We will be assuming that all the animations start from 0s for all values .. and the first index is actually the second frame for all animations
This should reduce the complexity for us

In [9]:
def process_track_to_dict(json_obj):
    tracks_list = json_obj[0]['tracks']
    tracks_dict = {}
    for elem in tracks_list:
        name = elem['name']
        times = elem['times']
        values = elem['values']
        tracks_dict[name] = {'times': times, 'values': values}
    json_obj[0]['tracks_dict'] = tracks_dict

In [10]:
def are_lists_equal(lst1, lst2):
    if len(lst1) != len(lst2): return False
    for elem1, elem2 in zip(lst1, lst2):
        if elem1 != elem2:
            return False
    return True

In [11]:
def _add_missing_categories(json1, json2):
    for name in json1[0]['tracks_dict']:
        if name not in json2[0]['tracks_dict']:
            json2[0]['tracks_dict'][name] = deepcopy( json1[0]['tracks_dict'][name] )
            for idx in range(len( json2[0]['tracks_dict'][name]['values'] )):
                json2[0]['tracks_dict'][name]['values'][idx] = 0

In [12]:
def add_missing_categories(json1, json2):
    _add_missing_categories(json1, json2)
    _add_missing_categories(json2, json1)

In [13]:
def assert_equal_times(json1, json2):
    for name in json1[0]['tracks_dict']:
        if name in json2[0]['tracks_dict']:
            assert are_lists_equal(json1[0]['tracks_dict'][name]['times'], json2[0]['tracks_dict'][name]['times'])

In [14]:
def preprocess_all_sounds(list_json_obj):
    [process_track_to_dict(json_obj) for json_obj in list_json_obj]
    for json1 in list_json_obj:
        for json2 in list_json_obj:
            add_missing_categories(json1, json2)
            assert_equal_times(json1, json2)

In [15]:
preprocess_all_sounds(list_json_obj=all_sounds)

In [16]:
def get_transition_from_json1_to_json2(json1, json2, transition_name: str):
    transition = {}
    transition['name'] = transition_name
    
    json1 = json1[0]
    json2 = json2[0]
    
    assert json1['duration'] == json2['duration']
    transition['duration'] = json1['duration']
    
    transition['tracks_dict'] = {}
    for name in json2['tracks_dict']:
        if not name in json1['tracks_dict']:
            assert False, name + ' in ' + transition_name
        
        elem1 = json1['tracks_dict'][name]
        elem2 = json2['tracks_dict'][name]
        
        transition['tracks_dict'][name] = {}
        
        assert are_lists_equal(elem1['times'], elem2['times'])
        transition['tracks_dict'][name]['times'] = elem1['times']
        
        vals1 = elem1['values']
        vals2 = elem2['values']
        
        vals1_reversed = reversed(vals1)
        transition_vals = [elem1 + elem2 for elem1, elem2 in zip(vals1_reversed, vals2)]
        transition['tracks_dict'][name]['values'] = transition_vals
    return [transition]

In [17]:
for name1, json1 in zip(sound_names, all_sounds):
    for name2, json2 in zip(sound_names, all_sounds):
        if name1 == name2:
            continue
        filename = name1 + '_to_' + name2
        transition_1_to_2 = get_transition_from_json1_to_json2(json1, json2, name1 + '_to_' + name2)
        with open(filename + '.json', 'w') as f:
            json.dump(transition_1_to_2, f)